# Cost of living & Venues Data Analysis of most popular cities in Bolivia

## 1. Introduction

### 1.1. Description & Disscusion of the Background

[Bolivia](https://en.wikipedia.org/wiki/Bolivia) according to the census conducted in 2012 has a population of 10,027,254 people, and the main cities according to the [population census](http://ibce.org.bo/publicaciones-descarga.php?id=2119&opcion=5) are:

| City |  Population   | Population Density |
|------|---------------|--------------------|
|  La Paz |  2.706.351 |      20.20         |
|  Santa Cruz |  2.706.351 |      7.16      |
|  Cochabamba |  2.706.351 |      31.60     |
|  Total      |  6.191.533 |          |

Only these 3 cities represent 61.74% of the general population, that is why the analysis will focus on these 3 cities in the following points:
*   Cost of living
*   Similarities between these 3 cities from the economic point of view
*   Relationship between the venues of each city through the Foursquare API to find similarities between the different districts of the city of La Paz, Santa Cruz and Cochabamba

### 1.2 Data Description

* For the cost of living analysis, [Numbeo](https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2010&region=005) data will be taken to determine the growth of the city of La Paz and a prediction model until the year 2025.
* For the analysis of the 3 cities, the detailed data of the [Numbeo website](https://www.numbeo.com/cost-of-living/in/Santa-Cruz-Bolivia) will be obtained, including the cost of basic basket items, among others, and the comparison will be established against the different capitals of South America and the data clustering will be carried out to determine a classification and similarity between the different cities.
* Finally, an analysis will be made between the different districts of the city of La Paz, Cochabamba and Santa Cruz to determine which districts are similar in relation to points of interest found through the [Foursquare API](https://developer.foursquare.com/).